# Algorítmo de limpieza de potencia aparente de Circuitos

El presente algorítmo permitirá obtener valores de potencia aparente validada para almacenarla en el DWH, la estampa de tiempo de la validación es de 1 minuto.

### Pasos a seguir durante el proceso:

 - Limpieza de outliers con el Método: DBSCAN
 - Aplicación del algorÍtmo de validación de máximos y mínimos
 - Almacenamiento de los datos validados en el DWH

### Limpieza de outliers con el Método: DBSCAN

Utilizaremos la biblioreca **sklearn** la cual tiene listo el algoritmo DBSCAN para ser implementado.

In [1]:
### Importación de clases y paquetes

from ExceptionManager import ExceptionManager
from HDFSContext import HDFSContext
from GenericDataFrame import GenericDataFrame
from DBContextDw import DBContextDw
from EtlDimensionAL import EtlDimensionAL
from Queries import Queries
from GetCircuitos import GetCircuitos
from Dbscan import Dbscan

from pyspark.sql.types import StructType,StructField,TimestampType,FloatType,StringType,IntegerType
from pyspark.sql.functions import when,date_format
import pyspark.sql.functions as func
from pyspark.sql.functions import to_timestamp, col, regexp_replace
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

#### Obtención de los datos de Circuitos desde el HDFS ( POTENCIA DESTINO)

In [2]:
# Datos de entrada

def Obtener_Circuitos(fecha_inicio,fecha_fin,circuito=None,destino=False):
    # Datos De circuitos (DataFrame Pyspark)
    getCircuitos = GetCircuitos(TableName = '')
    if(destino):
        circuitos = getCircuitos.GetDataDestino(fecha_inicio,fecha_fin,fileName='circuitos_destino_*',circuito=circuito)
    else:
        circuitos = getCircuitos.GetDataOrigen(fecha_inicio,fecha_fin,fileName='circuitos_origen_*',circuito=circuito)
    return circuitos

def Obtener_Calidad_Circuitos(fecha_inicio,fecha_fin,circuito=None,destino=False):
    # Datos De circuitos (DataFrame Pyspark)
    getCircuitos = GetCircuitos(TableName = '')
    if(destino):
        circuitos = getCircuitos.GetDataQualityDestino(fecha_inicio,fecha_fin,fileName='calidad_circuitos_destino_*',circuito=circuito)
    else:
        circuitos = getCircuitos.GetDataQualityOrigen(fecha_inicio,fecha_fin,fileName='calidad_circuitos_origen_*',circuito=circuito)
    return circuitos
    
def Transformar_Pandas(df_pyspark,column_index):
    # Datos a pandas dataframe
    datosDestino = circuitosDestino.toPandas().set_index(column_index)
    return datosDestino

In [3]:
# Proceso de Limpieza

def Limpiar_Datos(datosDestino):
    dbscan = Dbscan(datosDestino)
    dbscan.Limpiar_outliers()
    dbscan.Resumen_Datos()
    return dbscan.outliers,dbscan.datos_limpios

In [4]:
# Almacenar datos pre-procesados

def Almacenar_Datos_Sin_Outliers(df_pyspark,circuito):
    path = genericDataFrame.hdfsContext.HdfsPath('',circuito+'.json')
    print(path)
    df_pyspark.fillna('None').write.format('json').mode('overwrite').save(path)
    return True

In [5]:
#Método para limpiar los datos a nivel de mpa-reduce

def Limpiar(x):
    id = x[0]
    datos = list(x[1])
    salida = pd.DataFrame()
    ids = []
    fechas = []
    potencia = []
    calidad = []
    for i in range(0,len(datos)):
        ids.append(datos[i][0])
        fechas.append(datos[i][1])
        potencia.append(datos[i][2])
        calidad.append(datos[i][3])
    
    salida['Id'] = ids
    salida['Fecha'] = fechas
    salida['Potencia'] = potencia
    salida['Calidad'] = calidad
    
    outliers,datos_limpios = Limpiar_Datos(salida.set_index('Fecha'))
    
    return (id, [outliers,datos_limpios,salida])

In [6]:
# Metodo para completar los datos por Interpolación

def Completar_Datos(x,tipo,fecha_inicio,fecha_fin):
    id = x[0]
    datos = list(x[1])
    
    outliers = datos[0].reset_index()
    datos_limpios = datos[1].reset_index()
    datos_originales = datos[2]
    
    
    if(datos_limpios['Id'].count()==0):
        datos_limpios = outliers
    
    resultado = pd.merge(datos_originales,datos_limpios,on='Id')
    resultado = resultado[resultado['Calidad'].isin(['Normal','AL','AL.L5','AL.L6','L1'])]
    
    if(resultado['Id'].count()==0):
        resultado = pd.merge(datos_originales,datos_limpios,on='Id')
    
    
    puntoInicial = np.array([datetime.datetime(fecha_inicio.year,fecha_inicio.month,fecha_inicio.day,0,0,0) - timedelta(seconds=1)])
    puntoInicial = pd.DataFrame(puntoInicial,columns=['Fecha'])
    
    puntoFinal = np.array([datetime.datetime(fecha_inicio.year,fecha_inicio.month,fecha_inicio.day,23,59,59) + timedelta(seconds=1)])
    puntoFinal = pd.DataFrame(puntoFinal,columns=['Fecha'])

    resultado = resultado[['Id','Fecha_x','Potencia_x','Calidad']]\
    .rename(columns={'Fecha_x':'Fecha','Potencia_x':'Potencia'})
    
    maximo = resultado['Id'].values[-1]
    
    resultado = pd.merge(puntoInicial,resultado,on='Fecha',how='outer')
    resultado = pd.merge(puntoFinal,resultado,on='Fecha',how='outer')

    values = {'Fecha':datetime.datetime(1970,1,1,0,0,0),'Id':maximo,'Potencia':0,'Calidad':'Normal'}
    resultado = resultado.fillna(value=values)
    resultado = resultado.set_index('Fecha').resample('S').mean()
    resultado = resultado[1:-1]
    resultado = resultado.interpolate(method='values')
    resultado = resultado.interpolate(method='values',limit_direction='backward')
    
    resultado_segundo = resultado.copy()
    
    resultado = resultado.resample('T').max()
    resultado['Elemento'] = id
    resultado['Tipo'] = tipo

    if(tipo=='Origen'):
        calidad_circuitos = Obtener_Calidad_Circuitos(fecha_inicio,fecha_fin,circuito=id,destino=False)
    else:
        calidad_circuitos = Obtener_Calidad_Circuitos(fecha_inicio,fecha_fin,circuito=id,destino=True)
        
    calidad_circuitos = calidad_circuitos.toPandas().set_index('Fecha')[['Calidad','LimMaxOperacion',
                                                                         'LimOperacionContinuo','LimTermico',
                                                                         'TagCalidad','TagPotencia']]
    
    resultado = pd.merge(calidad_circuitos,resultado,on='Fecha')

    resultado_segundo = resultado_segundo.reset_index().groupby('Potencia')['Fecha'].min().reset_index()\
    .set_index('Potencia').rename(columns={'Fecha':'FechaMaximo'})
    resultado = resultado.reset_index().set_index('Potencia')
    resultado = pd.merge(resultado_segundo,resultado,on='Potencia')
    resultado = resultado.reset_index().set_index('Fecha').sort_index().reset_index()
    
    return (id,resultado)


In [7]:
#Algoritmo de validación de datos

def Algoritmo_Validacion_Datos(df,deltaDiffCircuito):         
    df_origen = df.filter(col('Tipo')=='Origen')\
    .select(col('Fecha').alias('FechaOrigen'),
            col('Potencia').alias('PotenciaOrigen'),
            col('Calidad').alias('CalidadOrigen'),
            col('LimMaxOperacion').alias('LimMaxOperacionOrigen'),
            col('LimOperacionContinuo').alias('LimOperacionContinuoOrigen'),
            col('LimTermico').alias('LimTermicoOrigen'),
            col('TagCalidad').alias('TagCalidadOrigen'),
            col('TagPotencia').alias('TagPotenciaOrigen'),
            col('Elemento').alias('ElementoOrigen'),
            col('FechaMaximo').alias('FechaMaximoOrigen'))\
    .withColumn('SuperiorOrigen',func.round(col('PotenciaOrigen')/col('LimTermicoOrigen'),5))\
    .withColumn('CargabilidadOrigen',func.round(col('PotenciaOrigen')/col('LimOperacionContinuoOrigen'),5))
    
    df_destino = df.filter(col('Tipo')=='Destino')\
    .select(col('Fecha').alias('FechaDestino'),
            col('Potencia').alias('PotenciaDestino'),
            col('Calidad').alias('CalidadDestino'),
            col('LimMaxOperacion').alias('LimMaxOperacionDestino'),
            col('LimOperacionContinuo').alias('LimOperacionContinuoDestino'),
            col('LimTermico').alias('LimTermicoDestino'),
            col('TagCalidad').alias('TagCalidadDestino'),
            col('TagPotencia').alias('TagPotenciaDestino'),
            col('Elemento').alias('ElementoDestino'),
            col('FechaMaximo').alias('FechaMaximoDestino'))\
    .withColumn('SuperiorDestino',func.round(col('PotenciaDestino')/col('LimTermicoDestino'),5).cast('double'))\
    .withColumn('CargabilidadDestino',func.round(col('PotenciaDestino')/col('LimOperacionContinuoDestino'),5).cast('double'))
    
    df = df_origen.join(df_destino,
                        (df_origen.FechaOrigen==df_destino.FechaDestino) &\
                        (df_origen.ElementoOrigen==df_destino.ElementoDestino))\
    .select(when(col('FechaOrigen').isNull(),col('FechaDestino')).otherwise(col('FechaOrigen')).alias('Fecha'),
            when(col('ElementoOrigen').isNull(),col('ElementoDestino')).otherwise(col('ElementoOrigen')).alias('Elemento'),
            when(col('LimOperacionContinuoOrigen').isNull(),col('LimOperacionContinuoDestino')).otherwise(col('LimOperacionContinuoOrigen')).alias('LimOperacionContinuo'),
            when(col('LimTermicoOrigen').isNull(),col('LimTermicoDestino')).otherwise(col('LimTermicoOrigen')).alias('LimTermico'),
            when(col('LimMaxOperacionOrigen').isNull(),col('LimMaxOperacionDestino')).otherwise(col('LimMaxOperacionOrigen')).alias('LimMaxOperacion'),
            
            when(col('PotenciaOrigen')>=col('PotenciaDestino'),
                 when((col('PotenciaOrigen')-col('PotenciaDestino'))<deltaDiffCircuito,col('TagPotenciaOrigen'))\
                 .otherwise(col('TagPotenciaDestino')))\
            .otherwise(col('TagPotenciaDestino')).alias('TagPotencia'),
            
            when(col('PotenciaOrigen')>=col('PotenciaDestino'),
                 when((col('PotenciaOrigen')-col('PotenciaDestino'))<deltaDiffCircuito,col('PotenciaOrigen'))\
                 .otherwise(col('PotenciaDestino')))\
            .otherwise(col('PotenciaDestino')).alias('Potencia'),
            
            when(col('PotenciaOrigen')>=col('PotenciaDestino'),
                 when((col('PotenciaOrigen')-col('PotenciaDestino'))<deltaDiffCircuito,col('SuperiorOrigen'))\
                 .otherwise(col('SuperiorDestino')))\
            .otherwise(col('SuperiorDestino')).alias('Superior'),
            
            when(col('PotenciaOrigen')>=col('PotenciaDestino'),
                 when((col('PotenciaOrigen')-col('PotenciaDestino'))<deltaDiffCircuito,col('CargabilidadOrigen'))\
                 .otherwise(col('CargabilidadDestino')))\
            .otherwise(col('CargabilidadDestino')).alias('Cargabilidad'),
            
            when(col('PotenciaOrigen')>=col('PotenciaDestino'),
                 when((col('PotenciaOrigen')-col('PotenciaDestino'))<deltaDiffCircuito,col('FechaMaximoOrigen'))\
                 .otherwise(col('FechaMaximoDestino')))\
            .otherwise(col('FechaMaximoDestino')).alias('FechaMaximo'),
            
            when(col('PotenciaOrigen')>=col('PotenciaDestino'),
                 when((col('PotenciaOrigen')-col('PotenciaDestino'))<deltaDiffCircuito,col('CalidadOrigen'))\
                 .otherwise(col('CalidadDestino')))\
            .otherwise(col('CalidadDestino')).alias('Calidad')).orderBy('Elemento','Fecha')
    
    return df
    
    

In [8]:
# Almacenamiento de los datos procesados

def Load_Upsert_data(transform_data,fecha_inicio,fecha_fin,accesoDatos):
    """Método que realiza la lógica de creación de queries para el método upsert. """
    result_transact = []
    
    # Eliminación de los datos antes de la carga respectiva
    result = accesoDatos.
    
    #
    transform_data_map = transform_data.rdd.map(lambda x: (x.agt_id_pk, [x.agt_empresa_id_bk, x.agt_empresa, 
                                                                         x.agt_region_id_bk, x.agt_region,
                                                                         x.agt_und_negocio_id_bk, x.agt_und_negocio,
                                                                         x.agt_clase_unegocio_id_bk, x.agt_clase_unegocio,
                                                                         x.agt_estacion_id_bk, x.agt_estacion,
                                                                         x.agt_tipo_estacion_id_bk, x.agt_tipo_estacion,
                                                                         x.agt_grupo_gen_id_bk, x.agt_grupo_gen,
                                                                         x.agt_voltaje_id_bk, x.agt_voltaje,
                                                                         x.agt_tipo_elemento_id_bk, x.agt_tipo_elemento,
                                                                         x.agt_elemento_id_bk, x.agt_elemento,
                                                                         x.agt_operacion_comercial,x.fecha_carga]))

    querys_data_insert = transform_data_map.map(lambda x: Queries.Upsert_Query_Dim_Agente(x))
    #for index in querys_data_insert.collect():
    #    print(index[0],index[1])
    for index in querys_data_insert.collect():
        res = []
        pk = index[0]
        query = index[1]

        result = self._accesoDatos.UpsertDimension(query)

        res.extend([pk,result])
        result_transact.append(res)            

    schema = StructType([
            StructField("pk", IntegerType(),False),
            StructField("Result", BooleanType(),False)
        ])

    result_transact =  self._genericDataFrame.spark.createDataFrame(result_transact,schema=schema)

    return result_transact

In [9]:
# Proceso de Consolidación de datos.

def Integridad_Datos(df,agentes,tiempo,hora,fecha_inicio,fecha_fin,accesoDatos):
    agentes = agentes.filter((col('agt_clase_unegocio_id_bk')=='TRA') & (col('agt_tipo_elemento_id_bk')==3))
    
    df = df\
    .join(tiempo, date_format(df.Fecha,'yyyyMMdd').cast('int')==tiempo.tmpo_id_pk)\
    .join(hora, date_format(df.Fecha,'HHmm').cast('smallint')==hora.hora_id_pk)\
    .join(agentes, df.Elemento == agentes.agt_elemento_id_bk,how='left')\
    .select(date_format(df.Fecha,'yyyyMMdd').cast('int').alias('crk_tmpo_id_fk'),
            date_format(df.Fecha,'HHmm').cast('smallint').alias('crk_hora_id_fk'),
            agentes.agt_id_pk.alias('crk_agt_id_fk'),
            df.Potencia.alias('crk_potencia_aparente'),
            df.Superior.alias('crk_superior'),
            df.Cargabilidad.alias('crk_cargabilidad'),
            df.LimTermico.alias('crk_limite_termico'),
            df.LimMaxOperacion.alias('crk_limite_max_operacion'),
            df.LimOperacionContinuo.alias('crk_limite_operacion_cont'),
            df.FechaMaximo.alias('crk_fecha_maximo'),
            df.TagPotencia.alias('crk_tag_potencia'),
            df.Calidad.alias('crk_calidad'))
    
    faltantes = df.filter(agentes.agt_id_pk.isNull())
    
    #if(~faltantes.rdd.isEmpty()):
    #    return None
    
    Load_Upsert_data(df,fecha_inicio,fecha_fin,accesoDatos)
    #df.show()

In [10]:
# Ejecución Automática

class LimpiezaBL():
    """Lógica de negocio para realizar la limpieza de datos de circuitos."""
    def __init__ (self):
        dbContext = DBContextDw(Database='dwh_sirio',urlDriver='/home/jovyan/work/postgresql-42.2.12.jar')
        self._accesoDatos = EtlDimensionAL(dbContext)
        self._genericDataFrame = GenericDataFrame(HDFSContext(Path='PI_INTEGRATOR',DataBase='TRANSMISION',Schema=''))
        self._sc = self._genericDataFrame.spark.sparkContext
        self._deltaDiffCircuito = 110
        
    def Ejecucion(self,fecha_inicio, fecha_fin):
        if(fecha_inicio is None or fecha_fin is None):
            fecha_inicio = datetime.datetime.combine(datetime.date.today(),datetime.datetime.min.time()) 
            fecha_fin = fecha_inicio + timedelta(seconds=86399)

        circuitos_origen = Obtener_Circuitos(fecha_inicio,fecha_fin,destino=False)
        circuitos_destino = Obtener_Circuitos(fecha_inicio,fecha_fin,destino=True)

        if((circuitos_origen.count()==0) or (circuitos_destino.count()==0)):
            print('No existe Datos a Procesar. '+str(datetime.datetime.now()))
            return False

        #Procesamos por cada circuito para limpiar - Datos Origen
        circuitos_map_origen = circuitos_origen.rdd.map(lambda x: (x.Circuito,[x.Id,x.Fecha,x.Potencia,x.Calidad]))
        circuitos_map_group_origen = circuitos_map_origen.groupByKey()
        circuitos_pandas_origen = circuitos_map_group_origen.map(lambda x: Limpiar(x))
        circuitos_pandas_origen = circuitos_pandas_origen.map(lambda x: Completar_Datos(x,'Origen',fecha_inicio,fecha_fin))

        #Procesamos por cada circuito para limpiar - Datos Destino
        circuitos_map_destino = circuitos_destino.rdd.map(lambda x: (x.Circuito,[x.Id,x.Fecha,x.Potencia,x.Calidad]))
        circuitos_map_group_destino = circuitos_map_destino.groupByKey()
        circuitos_pandas_destino = circuitos_map_group_destino.map(lambda x: Limpiar(x))
        circuitos_pandas_destino = circuitos_pandas_destino.map(lambda x: Completar_Datos(x,'Destino',fecha_inicio,fecha_fin))

        #Unimos ambos RDDs
        datos_completos = self._sc.union([circuitos_pandas_origen,circuitos_pandas_destino])
        datos_completos = datos_completos.map(lambda x: (x[1].to_numpy()))
        datos_completos = datos_completos.flatMap(lambda x: [(datetime.datetime.strptime(datetime.datetime.strftime(d[0],'%Y-%m-%d %H:%M:%S'),'%Y-%m-%d %H:%M:%S'),
                                                              d[1],
                                                              datetime.datetime.strptime(datetime.datetime.strftime(d[2],'%Y-%m-%d %H:%M:%S'),'%Y-%m-%d %H:%M:%S'),
                                                              d[3],d[4],d[5],d[6],d[7],d[8],d[9],
                                                              d[10]) for d in list(x)])

        schema = StructType([StructField('Fecha', TimestampType(), False),
                             StructField('Potencia', FloatType(), False),
                             StructField('FechaMaximo', TimestampType(), False),
                             StructField('Calidad', StringType(), False),
                             StructField('LimMaxOperacion', FloatType(), False),
                             StructField('LimOperacionContinuo', FloatType(), False),
                             StructField('LimTermico', FloatType(), False),
                             StructField('TagCalidad', StringType(), False),
                             StructField('TagPotencia', StringType(), False),
                             StructField('Elemento', StringType(), False),
                             StructField('Tipo', StringType(), False)])

        df = self._genericDataFrame.spark.createDataFrame(datos_completos,schema)
        df = Algoritmo_Validacion_Datos(df,self._deltaDiffCircuito)
        df_elementos = self._accesoDatos.GetAllData('cen_dws.dim_agente')
        df_tiempo = self._accesoDatos.GetAllData('cen_dws.dim_tiempo')
        df_horas = self._accesoDatos.GetAllData('cen_dws.dim_hora')
        
        Integridad_Datos(df,df_elementos,df_tiempo,df_horas,fecha_inicio,fecha_fin,self._accesoDatos)



In [11]:
fecha_inicio = datetime.datetime.strptime('2019-10-10 00:00', '%Y-%m-%d %H:%M')
fecha_fin = datetime.datetime.strptime('2019-10-10 23:59', '%Y-%m-%d %H:%M')
ejecucion = LimpiezaBL()
ejecucion.Ejecucion(fecha_inicio,fecha_fin)

+--------------+--------------+-------------+---------------------+------------+----------------+------------------+------------------------+-------------------------+-------------------+--------------------+-----------+
|crk_tmpo_id_fk|crk_hora_id_fk|crk_agt_id_fk|crk_potencia_aparente|crk_superior|crk_cargabilidad|crk_limite_termico|crk_limite_max_operacion|crk_limite_operacion_cont|   crk_fecha_maximo|    crk_tag_potencia|crk_calidad|
+--------------+--------------+-------------+---------------------+------------+----------------+------------------+------------------------+-------------------------+-------------------+--------------------+-----------+
|      20191010|           148|           29|             50.95693|     0.61766|         0.30883|              82.5|                    93.0|                    165.0|2019-10-10 01:48:15|C_AGOYAN138BANOS_...|     Normal|
|      20191010|           148|         1942|             50.95693|     0.61766|         0.30883|              82.5|

In [12]:
#dbscan.Dibujar_Resultados(plt, circuito = circuito,tag = tag)